In [ ]:
# Arjun Bhan
# AB5666

In [ ]:
!pip install portalocker
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 12.1 MB/s eta 0:00:00


In [ ]:
import argparse
import logging
import time
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torchtext
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import DATASETS
from torchtext.utils import download_from_url
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torchmetrics
import torch.optim as optim
SEED = 1

Short Question

Set up the optimization problem below where we take a random y of data and want theta to converge to this y.

In [ ]:
from torch._functorch.vmap import lazy_load_decompositions

torch.manual_seed(SEED)
y = torch.randn(1,3)

theta = torch.randn(1,3, requires_grad=True)


optimizer = torch.optim.SGD([theta], lr = 0.01)

for epoch in range(100):
  optimizer.zero_grad()

  loss = torch.norm(theta - y) ** 2

  print('Epoch:{} Loss: {}'.format(epoch, loss))

  loss.backward()

  optimizer.step()

print(y)
print(theta)
with torch.no_grad():
  loss = torch.norm(theta - y) ** 2
  assert (loss.item() - 0.0)**2 <= 0.001

Epoch:0 Loss: 0.5702157020568848
Epoch:1 Loss: 0.5476351380348206
Epoch:2 Loss: 0.5259487628936768
Epoch:3 Loss: 0.5051211714744568
Epoch:4 Loss: 0.48511847853660583
Epoch:5 Loss: 0.4659077227115631
Epoch:6 Loss: 0.44745776057243347
Epoch:7 Loss: 0.4297385513782501
Epoch:8 Loss: 0.41272082924842834
Epoch:9 Loss: 0.3963770866394043
Epoch:10 Loss: 0.38068053126335144
Epoch:11 Loss: 0.3656056225299835
Epoch:12 Loss: 0.35112762451171875
Epoch:13 Loss: 0.3372229337692261
Epoch:14 Loss: 0.32386893033981323
Epoch:15 Loss: 0.3110436797142029
Epoch:16 Loss: 0.29872632026672363
Epoch:17 Loss: 0.28689679503440857
Epoch:18 Loss: 0.2755356729030609
Epoch:19 Loss: 0.2646244764328003
Epoch:20 Loss: 0.25414538383483887
Epoch:21 Loss: 0.24408121407032013
Epoch:22 Loss: 0.23441559076309204
Epoch:23 Loss: 0.2251327484846115
Epoch:24 Loss: 0.21621745824813843
Epoch:25 Loss: 0.20765528082847595
Epoch:26 Loss: 0.1994321197271347
Epoch:27 Loss: 0.19153456389904022
Epoch:28 Loss: 0.183949813246727
Epoch:29 Lo

# Neural Text Classifier - Information

For this problem, you will build a basic Neural Text Classifier. The problem will take you through some of the steps needed to be done, including the preprocessing.


The at a high level, the idea of this model goes as follows.
- We are given a training set $\{(x^{(i)}, y^{(i)})\}_{i=1}^{N}$ where each $x^{(i)}$ is a sentence and $y^{(i)}$ is a class label.
- First, we need to loop over $\{x^{(i)}\}_{i=1}^{N}$ and get the Vocabulary, the number of unique words we see.
- Once we do this, we will express each word as a one-hot representation. To do this, we will use a mapping from a unique word to an integer. For example, "the" might get index 3 and if there are 10 words (in the entire Vocabulary) then "the" would have a vector representation $x_{the} = (0,0,1,0,0,0,0,0,0,0)$. There will be many words in this Vocabulary, over 13,000. For this example, each word is mapped to a unique integer.
- We will feed batches of data to the model and each batch will be transformed into a tensor with words each word transformed to its integer index in VOCAB below.
- For example, we might get [["the man walks"], ["this is a sentence"]] -> [["the", "man", "walks"], ["this", "is", "a", "sentence"]] -> [[1, 4, 5], [6, 7, 8, 15]]. It depends on what unique integer each word gets.
- Different sentences have different numbers of tokens but all batches need to be the same dimension (this is how PyTorch works), so we need a padding token. So, for example, if the batch size is B = 2 and we given two sentences like ["a b c", "a b c d e"] then as a tensor this will become [[1, 2, 3, 0, 0], [1, 2, 3, 4, 5]] and notice that we padded the first example so that the tensor is of dimension (2, 5) with M = 5. In some sense, in each batch we need to figure out the maximum number of tokens for an instance and pad each instance to have the same length as this longest instance. To do the above, use the [collate function](https://stackoverflow.com/questions/65279115/how-to-use-collate-fn-with-dataloaders). The idea here is that the Dataloader takes in raw data and the collate function is applied to this data, returning formatting tensors we can use later on in the optimization.
- After padding, we feed batches of data to the classifier, these are of dimension (B, M). For example, we have a batch size of 2 above and M = 5. This will depend on the batch but here the batch size is B.
- Once we feed in (B, M) data to the network, we rewrite this as (B, M, vocab_size) by using a one-hot representation for each word.
- Then, we do as it hints in the model's forward method. We first take an average agross all the M elements of each element of the batch to get a (B, vocab_size) tensor that represents each instance. We pass this tensor through linear layer and nonlinear layers as unusual. The model returns logits, without the Softmax applied. This is a multiclass classfication task.

Finally, we optimize the network and check it's train and validation set accuracies. We'll use both direct methods and torchmetrics to do this.

### Information (if interested in more)
- torchtext repo: https://github.com/pytorch/text/tree/main/torchtext
- torchtext documentation: https://pytorch.org/text/stable/index.html
- collate function: https://stackoverflow.com/questions/65279115/how-to-use-collate-fn-with-dataloaders
- embedding layer: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

### Constants

In [ ]:
DATASET = "AG_NEWS"
DATA_DIR = ".data"
DEVICE = "cuda:0"
LR = 8.0
BATCH_SIZE = 16
NUM_EPOCHS = 5
MIN_FREQUENCY = 20
PADDING_VALUE = 0
PADDING_IDX = PADDING_VALUE

SEED = 1

# Get the tokenizer

In [ ]:
basic_english_tokenizer = get_tokenizer("basic_english")

In [ ]:
basic_english_tokenizer("This is some text ...")

['this', 'is', 'some', 'text', '.', '.', '.']

In [ ]:
TOKENIZER = basic_english_tokenizer

### Get the data and get the vocabulary.

In [ ]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield basic_english_tokenizer(text)

In [ ]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")

In [ ]:
for y, x in train_iter:
  print(y, x)
  break

3 Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


In [ ]:

VOCAB = build_vocab_from_iterator(
    yield_tokens(train_iter),
    min_freq = MIN_FREQUENCY,
    specials=('<pad>', '<unk>')
)

VOCAB.set_default_index(VOCAB['<unk>'])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [ ]:
assert VOCAB['<unk>'] == 1

Examples

In [ ]:
VOCAB['yoyooyoyoy'], VOCAB['house'], VOCAB['<pad>'], VOCAB['<unk>']

(1, 437, 0, 1)

In [ ]:
print(len(VOCAB))

13798


In [ ]:
VOCAB(TOKENIZER("House house houses ThisisnotaKNownWord"))

[437, 437, 4548, 1]

### Helper functions

In [ ]:
from torchtext.vocab.vocab_factory import Vocab

def text_pipeline(x):
    tokensX = basic_english_tokenizer(x)

    return VOCAB(tokensX)


def label_pipeline(x):
    return int(x) - 1

Nice link on collate_fn and DataLoader in PyTorch: https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

In [ ]:

def collate_batch(batch):
    label_list, text_list = [], []
    for (label, text) in batch:
        label_list.append(label_pipeline(label))


        processed_text = text_pipeline(text)
        processed_text = torch.tensor(processed_text, dtype=torch.int64)
        text_list.append(torch.tensor(processed_text))

    label_list = torch.tensor(label_list, dtype=torch.int64)


    text_list = pad_sequence(text_list, batch_first=True, padding_value=0)

    return label_list.to(DEVICE), text_list.to(DEVICE)

### Get the data

In [ ]:
train_iter = DATASETS[DATASET](root=DATA_DIR, split="train")
train_iter_set = set(train_iter)
arrayObject = []
for i in train_iter_set:
  if i[0] not in arrayObject:
    arrayObject.append(i[0])
num_class = len(arrayObject)
print(f"The number of classes is {num_class} ...")

The number of classes is 4 ...


In [ ]:
arrayObject = []
for i in train_iter_set:
  if i[0] not in arrayObject:
    arrayObject.append(i[0])
arrayObject

[1, 2, 3, 4]

In [ ]:
train_iter_set

{(1,
  "Golf: Record earnings for Singh  Vijay Singh leads from start to finish to win the 84 Lumber Classic  and eclipse Tiger Woods' season earnings record."),
 (2,
  'Uncharted territory Jets players were already thinking ahead to their showdown in New England next weekend before theyd toweled off from their showers after dodging a major disaster Sunday.'),
 (3,
  'Oil  amp; Natural Gas Buys 55 Stake in Australian Block (Update1) Oil  amp; Natural Gas Corp., India #39;s biggest oil producer, bought 55 percent of an oil exploration area off the coast of Western Australia from Calgary-based Antrim Energy Inc.'),
 (2,
  'Spadea Returns from the Brink  LYON, France (Reuters) - Seventh seed Vincent Spadea came  back from the brink to defeat Jurgen Melzer 1-6, 6-1, 7-6 at  the Lyon Grand Prix Friday.'),
 (2,
  'Hanson chasing number six Australian swimmer Brooke Hanson will be chasing her sixth gold medal at the World Short Course Championships in Indianapolis later this morning.'),
 (1,


### Set up the model

In [ ]:
class OneHotTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, num_class):
        super(OneHotTextClassificationModel, self).__init__()
        self.vocab_size = vocab_size
        self.num_class = num_class

        self.fc1 = nn.Linear(vocab_size, 100, bias = False)

        self.e = nn.Embedding(vocab_size, 100)


        self.fc2 = nn.Linear(100, self.num_class, bias = False)
        self.init_weights()

        self.use_embedding_layer = False

    def init_weights(self):
        self.fc1.weight.data = self.e.weight.data.clone().T
        initrange = 0.5
        self.fc2.weight.data.uniform_(-1  * initrange, initrange)

    def forward(self, x):
        B, K = x.shape

        if not self.use_embedding_layer:
          x = F.one_hot(x, num_classes = self.vocab_size).float()
          assert(x.shape == (B, K, self.vocab_size))

          x = self.fc1(x)
          assert(x.shape == (B, K, 100))
        else:
          x = self.e(x)
          assert(x.shape == (B, K, 100))

        x = torch.mean(x, dim = 1)
        assert(x.shape == (B, 100))

        x = torch.nn.ReLU()(x)
        assert(x.shape == (B, 100))

        x = self.fc2(x)
        assert(x.shape == (B, num_class))

        return x

In [ ]:
torch.manual_seed(SEED)

### Set up the data

In [ ]:
train_iter, test_iter = DATASETS[DATASET]()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size = BATCH_SIZE, shuffle = True, collate_fn = collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size = BATCH_SIZE, shuffle = False, collate_fn = collate_batch)
test_dataloader =  DataLoader(split_valid_, batch_size = BATCH_SIZE, shuffle = False, collate_fn = collate_batch)

### Train the model

In [ ]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_acc, total_count = 0, 0
    total_loss = 0.0
    log_interval = 200

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()

        predicted_label = model(text)

        loss = loss_fn(input=predicted_label, target=label)

        with torch.no_grad():
          total_loss += loss.item() * label.size(0)

        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), .1)

        optimizer.step()

        total_acc += (predicted_label.argmax(1) == label).sum().item()

        total_count +=  label.size(0)
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f} "
                "| loss {:8.3f}".format(
                    epoch, idx,
                    len(dataloader),
                    total_acc/total_count,
                    total_loss/total_count
                    )
            )
            total_acc, total_count, total_loss = 0, 0, 0.0

In [ ]:
def evaluate(dataloader, model):
    model.eval()
    accuracy_fn = torchmetrics.Accuracy(num_classes = num_class, task = 'multiclass').to(DEVICE)
    total_acc = 0.0
    total_count = 0.0
    total_acc_argmax = 0
    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            total_count += label.size(0)

            numCorrect = accuracy_fn(predicted_label, label).detach().cpu().numpy() * label.size(0)
            total_acc += numCorrect
            assert(numCorrect == (predicted_label.argmax(1) == label).sum())

    accuracy = total_acc/ total_count
    return accuracy

# Train the model

You should get an accuracy > 80% for the training set. This might take quite a bit of time to run since we use one-hot. Use nn.Embedding if you want to check this quickly. You should get the SAME answer using either method.

In [ ]:

loss_fn = torch.nn.CrossEntropyLoss().to(DEVICE)


model = OneHotTextClassificationModel(len(VOCAB), num_class).to(DEVICE)
# Instantiate the SGD optimizer with parameters LR
optimizer = optim.SGD(model.parameters(), lr = LR)

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, loss_fn, epoch)

    accu_val = evaluate(valid_dataloader, model)
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch,
            time.time() - epoch_start_time,
            accu_val
            )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader, model)
print("test accuracy {:8.3f}".format(accu_test))

<ipython-input-18-ca896283e932>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_list.append(torch.tensor(processed_text))


| epoch   1 |   200/ 7125 batches | accuracy    0.322 | loss    1.424
| epoch   1 |   400/ 7125 batches | accuracy    0.422 | loss    1.279
| epoch   1 |   600/ 7125 batches | accuracy    0.444 | loss    1.244
| epoch   1 |   800/ 7125 batches | accuracy    0.497 | loss    1.179
| epoch   1 |  1000/ 7125 batches | accuracy    0.531 | loss    1.127
| epoch   1 |  1200/ 7125 batches | accuracy    0.529 | loss    1.128
| epoch   1 |  1400/ 7125 batches | accuracy    0.530 | loss    1.116
| epoch   1 |  1600/ 7125 batches | accuracy    0.579 | loss    1.025
| epoch   1 |  1800/ 7125 batches | accuracy    0.571 | loss    1.038
| epoch   1 |  2000/ 7125 batches | accuracy    0.603 | loss    0.962
| epoch   1 |  2200/ 7125 batches | accuracy    0.615 | loss    0.960
| epoch   1 |  2400/ 7125 batches | accuracy    0.605 | loss    1.010
| epoch   1 |  2600/ 7125 batches | accuracy    0.644 | loss    0.922
| epoch   1 |  2800/ 7125 batches | accuracy    0.639 | loss    0.912
| epoch   1 |  3000/